In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [2]:
data = pd.read_csv('data/train.csv')

In [3]:
data.head(10)

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,Clayey,Sugarcane,36,4,5,28-28
1,1,27,69,65,Sandy,Millets,30,6,18,28-28
2,2,29,63,32,Sandy,Millets,24,12,16,17-17-17
3,3,35,62,54,Sandy,Barley,39,12,4,10-26-26
4,4,35,58,43,Red,Paddy,37,2,16,DAP
5,5,30,59,29,Red,Pulses,10,0,9,20-20
6,6,27,62,53,Sandy,Paddy,26,15,22,28-28
7,7,36,62,44,Red,Pulses,30,12,35,14-35-14
8,8,36,51,32,Loamy,Tobacco,19,17,29,17-17-17
9,9,28,50,35,Red,Tobacco,25,12,16,20-20


In [4]:
soil_encoder = LabelEncoder()
crop_encoder = LabelEncoder()
ferti_encoder = LabelEncoder()
scaler = MinMaxScaler()

In [5]:
data['Soil Type'] = soil_encoder.fit_transform(data['Soil Type'])
data['Crop Type'] = crop_encoder.fit_transform(data['Crop Type'])
data['Fertilizer Name'] = ferti_encoder.fit_transform(data['Fertilizer Name'])
data.head(10)

,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous,Fertilizer Name
0,0,37,70,36,1,8,36,4,5,4
1,1,27,69,65,4,4,30,6,18,4
2,2,29,63,32,4,4,24,12,16,2
3,3,35,62,54,4,0,39,12,4,0
4,4,35,58,43,3,6,37,2,16,5
5,5,30,59,29,3,7,10,0,9,3
6,6,27,62,53,4,6,26,15,22,4
7,7,36,62,44,3,7,30,12,35,1
8,8,36,51,32,2,9,19,17,29,2
9,9,28,50,35,3,9,25,12,16,3


In [6]:
ferti_col = data['Fertilizer Name']
no_ferti_data = data.drop(columns=['Fertilizer Name'])
data = pd.concat([ ferti_col, no_ferti_data], axis=1)
data.head(10)

,Fertilizer Name,id,Temparature,Humidity,Moisture,Soil Type,Crop Type,Nitrogen,Potassium,Phosphorous
0,4,0,37,70,36,1,8,36,4,5
1,4,1,27,69,65,4,4,30,6,18
2,2,2,29,63,32,4,4,24,12,16
3,0,3,35,62,54,4,0,39,12,4
4,5,4,35,58,43,3,6,37,2,16
5,3,5,30,59,29,3,7,10,0,9
6,4,6,27,62,53,4,6,26,15,22
7,1,7,36,62,44,3,7,30,12,35
8,2,8,36,51,32,2,9,19,17,29
9,3,9,28,50,35,3,9,25,12,16


In [7]:
data = data.drop(columns = 'id')
# data = scaler.fit_transform(data)

In [8]:
data = np.array(data)
data

array([[ 4, 37, 70, ..., 36,  4,  5],
       [ 4, 27, 69, ..., 30,  6, 18],
       [ 2, 29, 63, ..., 24, 12, 16],
       ...,
       [ 0, 35, 68, ...,  6, 11, 29],
       [ 3, 31, 68, ...,  9, 11, 12],
       [ 6, 33, 55, ...,  4, 10,  9]])

In [9]:
m, n = data.shape
print(m, n)

750000 9


In [10]:
np.random.shuffle(data)
data

array([[ 4, 27, 61, ..., 12, 19, 34],
       [ 5, 31, 55, ...,  6,  8, 16],
       [ 1, 31, 53, ..., 30, 12, 21],
       ...,
       [ 2, 35, 51, ...,  6,  6, 15],
       [ 6, 26, 53, ..., 36,  8, 18],
       [ 4, 35, 58, ..., 16, 11, 19]])

In [11]:
data_dev = data[0:50000].T 
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = scaler.fit_transform(X_dev)


In [12]:
data_train = data[50000:m].T 
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = scaler.fit_transform(X_train)

In [13]:
n_train, m_train = X_train.shape
m_train

700000

In [ ]:
def init_params():
    W1 = np.random.rand(256, 8) - 0.5
    b1 = np.random.rand(256, 1) - 0.5
    W2 = np.random.rand(7, 256) - 0.5
    b2 = np.random.rand(7, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(0, Z)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  # for numerical stability
    return expZ / np.sum(expZ, axis=0, keepdims=True)

def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_der(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T 
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_der(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2


In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions=predictions, Y=Y))
        
    return W1, b1, W2, b2 

In [17]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 600)

Iteration:  0
[4 4 4 ... 4 4 4] [3 1 1 ... 2 6 4]
0.14780714285714286
Iteration:  10
[1 4 4 ... 1 6 4] [3 1 1 ... 2 6 4]
0.14629714285714285
Iteration:  20
[1 1 4 ... 1 6 4] [3 1 1 ... 2 6 4]
0.14697
Iteration:  30
[1 1 3 ... 1 3 4] [3 1 1 ... 2 6 4]
0.14821571428571428
Iteration:  40
[1 1 3 ... 1 3 0] [3 1 1 ... 2 6 4]
0.1480642857142857
Iteration:  50
[1 1 3 ... 1 3 0] [3 1 1 ... 2 6 4]
0.1482257142857143
Iteration:  60
[1 1 3 ... 1 3 0] [3 1 1 ... 2 6 4]
0.14832142857142858
Iteration:  70
[1 1 3 ... 1 3 0] [3 1 1 ... 2 6 4]
0.14848285714285714
Iteration:  80
[1 1 3 ... 1 3 0] [3 1 1 ... 2 6 4]
0.14868285714285714
Iteration:  90
[1 1 3 ... 1 3 0] [3 1 1 ... 2 6 4]
0.14863
Iteration:  100
[1 1 1 ... 1 3 0] [3 1 1 ... 2 6 4]
0.14889285714285713
Iteration:  110
[1 1 1 ... 1 3 0] [3 1 1 ... 2 6 4]
0.1490257142857143
Iteration:  120
[1 1 1 ... 1 3 0] [3 1 1 ... 2 6 4]
0.14901857142857142
Iteration:  130
[1 1 1 ... 1 3 0] [3 1 1 ... 2 6 4]
0.14915
Iteration:  140
[1 1 1 ... 1 3 0] [3 1 1 .

KeyboardInterrupt: 